# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [ ]:
init_aviat_df = pd.read_csv('AviationData.csv', encoding='latin-1', low_memory=False)

In [ ]:
init_aviat_df.head()

In [ ]:
init_aviat_df.info()

In [ ]:
init_aviat_df.describe(include = 'object')

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [ ]:
print(init_aviat_df['Aircraft.Category'].isna().sum())
print(init_aviat_df['Aircraft.Category'].value_counts())

In [ ]:
init_aviat_df.head()[['Make', 'Aircraft.Category']]

In [ ]:
init_aviat_df['Aircraft.Category'].fillna('Airplane', inplace = True)
init_aviat_df['Aircraft.Category'].value_counts()

In [ ]:
air_df = init_aviat_df[init_aviat_df['Aircraft.Category'] == 'Airplane']
air_df.info()

In [ ]:
air_df['Amateur.Built'].value_counts()

In [ ]:
# filter amateur builds
air_df = air_df[air_df['Amateur.Built'] == 'No']

In [ ]:
air_df = air_df.dropna(subset=['Publication.Date']) 
air_df['Report_Date'] = pd.to_datetime(air_df['Publication.Date']) 
air_df = air_df.sort_values(by='Report_Date') 
air_df = air_df[air_df['Report_Date'] > '1983']

In [ ]:
air_df.head()

In [ ]:
air_df['Report_Date']

In [ ]:
print(air_df['Report_Date'].min(), air_df['Report_Date'].max())

Why is it only showing a small amount of data from 1983 to 2022? CSV correctly downloaded and showing all dates. 

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [ ]:
# identify injury-related columns by searching for 'njur' in the column names
# this matches columns like 'Total.Fatal.Injuries', 'Total.Serious.Injuries'
# we skip the first match with [1::] assuming the first one may be a duplicate or not relevant for summing
injury_col_names = air_df.columns[air_df.columns.str.contains('njur')] #[1::]

# display the selected columns to verify they are related to injury counts
print(injury_col_names)

# count missing (NaN) values in each of the selected injury-related columns
# this helps assess how much data cleaning or imputation is needed
air_df[injury_col_names].isna().sum()



In [ ]:
air_df['Injury.Severity'].value_counts()

In [ ]:
# check how many rows have all injury-related columns missing
# these rows likely have no usable injury data and may not be useful for injury analysis
# rows with no data in any injury columns provide no value for injury metrics and can be removed
air_df[injury_col_names].isna().all(axis = 1).sum()


In [ ]:
# remove rows where all injury-related columns are missing
# rhese rows do not contribute any injury information and would skew injury-related metrics if kept
# if there is no injury data at all for an event, it is not useful for severity analysis
air_df = air_df[~air_df[injury_col_names].isna().all(axis=1)]

In [ ]:
# after removing fully-empty injury rows, count the remaining missing values in each injury-related column
# this helps assess the completeness of injury data and informs whether further imputation or filtering is needed
# the remaining missing values might be handled later depending on how critical each injury type is to the analysis
air_df[injury_col_names].isna().sum()


In [ ]:
# fill missing values in all injury-related columns with 0
# if injury data is missing, we assume no injuries were reported
air_df.loc[:, injury_col_names] = air_df[injury_col_names].fillna(0)

# confirm that there are no more missing values in injury-related columns
air_df[injury_col_names].isna().sum()


In [ ]:
# convert injury columns to numeric, forcing invalid entries to NaN
air_df[injury_col_names] = air_df[injury_col_names].apply(pd.to_numeric, errors='coerce')

# calculate the total number of passengers involved in the accident
air_df['N_passenger'] = air_df[injury_col_names].sum(axis=1)

# remove rows with total passengers == 0
air_df = air_df[air_df['N_passenger'] > 0]

# calculate the serious injury fraction
air_df['ser_inj_frac'] = (
    air_df['Total.Fatal.Injuries'] + air_df['Total.Serious.Injuries']
) / air_df['N_passenger']


In [ ]:
air_df.head()

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [ ]:
air_df.info()

In [ ]:
print(air_df['Aircraft.damage'].unique())
print(air_df['Aircraft.damage'].value_counts())

In [ ]:
air_df['Aircraft.damage'] = air_df['Aircraft.damage'].replace({'Unknown': np.nan})
air_df.dropna(subset=['Aircraft.damage'], inplace=True)
air_df.info()

In [ ]:
# create a new column 'is_destroyed' that flags whether the aircraft was destroyed
# this creates a boolean mask where 'Destroyed' = True, then converts it to int (1 for destroyed, 0 otherwise)
air_df['is_destroyed'] = (air_df['Aircraft.damage'] == 'Destroyed').astype('int')

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [ ]:
# convert all entries in 'Make' to uppercase
# this ensures consistent casing (e.g., 'Cessna' and 'CESSNA' are treated as the same)
air_df['Make'] = air_df['Make'].str.upper()

# remove leading and trailing whitespace
# this cleans up any extra spaces that may have been typed into the original data
air_df['Make'] = air_df['Make'].str.strip()

# replace ambiguous or missing values like 'UNKNOWN' or empty strings with NaN
# these values do not provide useful information and will be removed
air_df['Make'] = air_df['Make'].replace({'UNKNOWN': np.nan, '': np.nan})

# drop rows where 'Make' is now missing (NaN)
# these rows cannot be used for manufacturer-level analysis
air_df.dropna(subset=['Make'], inplace=True)

# standardize known variations of the same manufacturer
# this helps combine entries that should be grouped under one manufacturer
air_df['Make'] = air_df['Make'].replace({'CESSNA AIRCRAFT': 'CESSNA','CESSNA AIR': 'CESSNA','PIPER AIRCRAFT': 'PIPER'})

# filter to only include makes that appear at least 50 times
# this reduces noise from extremely rare or one-off manufacturers
make_counts = air_df['Make'].value_counts()
common_makes = make_counts[make_counts >= 50].index
air_df = air_df[air_df['Make'].isin(common_makes)]

air_df['Make'].head()         
air_df['Make'].value_counts() 

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [ ]:
air_df.dropna(subset=['Model'], inplace=True)

# as an example both beech and aero-commander have a model 100
air_df.groupby(['Model', 'Make']).count().loc['100']

In [ ]:
air_df['make_model'] = air_df['Make'] + '_' + air_df['Model'].str.upper()


In [ ]:
air_df.head()

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [ ]:
# preview the raw value counts for the 'Engine.Type' column
print(air_df['Engine.Type'].value_counts())

# replace placeholder or ambiguous values like 'UNKNOWN' or 'UNK' with NaN
# this prevents skewing analysis with unclear categories
air_df['Engine.Type'].replace({'Unknown': np.nan, 'UNK': np.nan}, inplace=True)

# we don't want any unknown categories computed in our statistics
# replacing them with NaNs is appropriate and ensures our summaries focus on meaningful values
# no need to fill these NaNs unless needed later for modeling

# remove engine types that occur only once — they can't support statistical claims
filtered_enginetype = air_df['Engine.Type'].value_counts()[air_df['Engine.Type'].value_counts() > 1]
air_df = air_df[air_df['Engine.Type'].isin(filtered_enginetype.index)]

# show the cleaned value counts for engine types
air_df['Engine.Type'].value_counts()

In [ ]:
# print unique values before cleaning to identify unknowns or formatting issues
print(air_df['Weather.Condition'].unique())

# replace ambiguous 'UNK', 'Unk' with NaN to avoid skewing statistics
air_df['Weather.Condition'].replace({'UNK': np.nan, 'Unk': np.nan}, inplace=True)

# print value counts to verify successful replacement and inspect distribution
print(air_df['Weather.Condition'].value_counts())

In [ ]:
# print how many times each engine count appears to identify potential issues (e.g., 0 engines)
print(air_df['Number.of.Engines'].value_counts())

# remove rows where number of engines is zero — likely invalid for flight analysis
air_df = air_df[air_df['Number.of.Engines'] > 0.0]

# confirm only valid unique values remain after cleaning
print(air_df['Number.of.Engines'].unique())

In [ ]:
# print the unique values in 'Purpose.of.flight' to check for placeholder categories
print(air_df['Purpose.of.flight'].unique())

# replace 'Unknown' entries with NaN so they don't affect statistical summaries or grouping
air_df['Purpose.of.flight'].replace({'Unknown': np.nan}, inplace=True)

# recheck unique values to confirm 'Unknown' has been successfully removed
print(air_df['Purpose.of.flight'].unique())

In [ ]:
# print the unique values in 'Broad.phase.of.flight' to inspect placeholder or unclear entries
print(air_df['Broad.phase.of.flight'].unique())

# replace placeholder categories like 'Unknown' and 'Other' with NaN
# This prevents vague categories from skewing analysis
air_df['Broad.phase.of.flight'].replace({'Unknown': np.nan, 'Other': np.nan}, inplace=True)

# display cleaned value counts to verify the result
print(air_df['Broad.phase.of.flight'].value_counts())

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [ ]:
air_df.info()

In [ ]:
# some valid column removals are 'FAR.Description', 'Air.carrier', and 'Schedule'
# these columns may contain too many missing values or offer little analytical value for the problem at hand
air_df = air_df.drop(columns=['FAR.Description', 'Air.carrier', 'Schedule'])
print(air_df.columns.tolist())

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [ ]:
air_df.to_csv('cleaned_aviation_data.csv', index=False)

#### 